In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [2]:
import torch

from utils import set_seed, print_trainable_parameters

from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          BitsAndBytesConfig, Trainer,
                          TrainingArguments, DataCollatorForLanguageModeling)

from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel

/home/nlplab11/anaconda3/envs/cdl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/nlplab11/anaconda3/envs/cdl/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/nlplab11/anaconda3/envs/cdl/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/nlplab11/anaconda3/envs/cdl/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [4]:
# BitsAndBytes Config for QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
checkpoint = 'beomi/kollama-13b'

In [6]:
# LOAD TOKENIZER
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    use_fast=False,
    truncation_side='left',
    )

In [7]:
model_checkpoint = 'checkpoint-240'

In [8]:
model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                            quantization_config=bnb_config,
                                            device_map='auto')

# PREPARE MODEL FOR K BIT TRAINING
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# LoRA Config
config = LoraConfig.from_json_file(f'outputs/{model_checkpoint}/adapter_config.json')

model = PeftModel.from_pretrained(model, model_id=f'outputs/{model_checkpoint}/')

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 28/28 [00:05<00:00,  4.91it/s]


In [9]:
!nvidia-smi

Wed Jul  5 10:44:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 86%   71C    P2   342W / 350W |  23079MiB / 24268MiB |     94%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:21:00.0 Off |                  N/A |
| 39%   

In [10]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(52000, 5120, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
              (v

In [24]:
prompt = [
    'Below is an instruction that describes a task, paired with an input that provides further context.',
    '아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n',
    'Write a response that appropriately completes the request.',
    '요청을 적절히 완료하는 응답을 작성하세요.\n',
    '### Instruction(명령어):',
    '아이의 요구사항을 그림 생성을 위한 프롬프트를 번역하세요.\n',
    '### Input(입력):',
    '아이: 17세기 도둑이 물 속에 다친 회색 말을 보며 슬퍼하는 모습을 담은 그림을 그려봐. 그리고 찬 영국 공기에 바람에 흔들리는 갈대와 망토를 넣어줘.\n',
    '### Response(응답):'
    ]

inputs = tokenizer('\n'.join(prompt), return_tensors="pt")

In [25]:
print('\n'.join(prompt))

Below is an instruction that describes a task, paired with an input that provides further context.
아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.

Write a response that appropriately completes the request.
요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
아이의 요구사항을 그림 생성을 위한 프롬프트를 번역하세요.

### Input(입력):
아이: 17세기 도둑이 물 속에 다친 회색 말을 보며 슬퍼하는 모습을 담은 그림을 그려봐. 그리고 찬 영국 공기에 바람에 흔들리는 갈대와 망토를 넣어줘.

### Response(응답):


In [26]:
with torch.no_grad():
    outputs = model.generate(input_ids=inputs['input_ids'].to('cuda'),
                             max_length=256,
                             no_repeat_ngram_size=2,
                             early_stopping=True)
    outputs = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    print(outputs)

Below is an instruction that describes a task, paired with an input that provides further context.
아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.

Write a response that appropriately completes the request.
요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
아이의 요구사항을 그림 생성을 위한 프롬프트를 번역하세요.

### Input(입력):
아이: 17세기 도둑이 물 속에 다친 회색 말을 보며 슬퍼하는 모습을 담은 그림을 그려봐. 그리고 찬 영국 공기에 바람에 흔들리는 갈대와 망토를 넣어줘.

### Response(응답):Wide-Angle Art a 16th century painting of a 17thcentury man with a fractured white hair, she is sleeping in a dark woods, 
